# Step2- Simple Vectorization

Word Embedding is one such technique where we can represent the text using vectors. The more popular forms of word embeddings are:

1. BoW (Bag of Words)
2. TF-IDF (Term Frequency-Inverse Document Frequency)

Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

In [ ]:
!pip install -U spacy==3.*
!python -m spacy download en_core_web_sm
!python -m spacy info

# A) Basic Bag-of-Words (BOW)

Bag of words is a Natural Language Processing technique of text modelling. In technical terms, we can say that it is a method of feature extraction with text data. This approach is a simple and flexible way of extracting features from documents.

A bag of words is a representation of text that describes the occurrence of words within a document. We just keep track of word counts and disregard the grammatical details and the word order. It is called a “bag” of words because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [ ]:
import spacy

from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 1) Plain frequency BOW

In [ ]:
# A corpus of sentences.
corpus = [
  "Red Bull drops hint on F1 engine.",
  "Honda exits F1, leaving F1 partner Red Bull.",
  "Hamilton eyes record eighth F1 title.",
  "Aston Martin announces sponsor."
]
corpus

We want to build a basic bag-of-words (BOW) representation of our corpus. Based on what you now know from the lesson, you can probably do this from scratch using dictionaries and lists (and maybe that's a good exercise). Fortunately, there are robust libraries which make it easy.

We can use the scikit-learn **CountVectorizer** which takes a collection of text documents and creates a matrix of token counts:<br>
https://scikit-learn.org/stable/index.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html




In [ ]:
vectorizer = CountVectorizer()
vectorizer

The *fit_transform* method does two things:
1. It learns a vocabulary dictionary from the corpus.
2. It returns a matrix where each row represents a document and each column represents a token (i.e. term).<br>

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit_transform


In [ ]:
bow = vectorizer.fit_transform(corpus)
bow

We can take a look at the features and vocabulary dictionary.

`Note-`**The CountVectorizer took care of tokenization and also removed punctuation and lower-cased everything.**

In [ ]:
# View features (tokens).
print(vectorizer.get_feature_names_out())

In [ ]:
# View vocabulary dictionary.
vectorizer.vocabulary_

Specifically, the **CountVectorizer** generates a sparse matrix using an efficient, compressed representation. The sparse matrix object includes a number of useful methods:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

In [ ]:
print(type(bow))

If we look at the raw structure, we'll see tuples where the first element represents the document, and the second element represents a token ID. It's then followed by a count of that token. So in the second document (index 1), token 8 ("f1") occurs twice.

In [ ]:
print(bow)

Before we explore further, we want to make a few modifications.
1. What if we want to use another tokenizer like spaCy's?
2. Instead of frequency, what if we want to have a binary BOW?


## 2) Binary BOW with custom tokenizer

**CountVectorizer** supports using a custom tokenizer. For every document, it will call your tokenizer and expect a list of tokens returned. We'll create a simple callback below which has spaCy tokenize and filter tokens, and then return them.

In [ ]:
# As usual, we start by importing spaCy and loading a statistical model.
nlp = spacy.load('en_core_web_sm')
nlp

In [ ]:
# Create a tokenizer callback using spaCy under the hood. Here, we tokenize
# the passed-in text and return the tokens, filtering out punctuation.
def spacy_tokenizer(doc):
  return [t.text for t in nlp(doc) if not t.is_punct]

spacy_tokenizer

This time, we instantiate **CountVectorizer** with our custom tokenizer (*spacy_tokenizer*), turn off case-folding, and also set the *binary* parameter to *True* so we simply get 1s and 0s marking token presence rather than token frequency.

In [ ]:
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, lowercase=False, binary=True)
vectorizer

In [ ]:
bow = vectorizer.fit_transform(corpus)
bow

Looking at the resulting feature names and vocabulary dictionary, we can see our *spacy_tokenizer* being used. If you're not convinced, you can remove the punctuation filtering in our tokenizer and rerun the code.

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
vectorizer.vocabulary_

To get a dense array representation of our sparse matrix, use *toarray*.<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.toarray.html#scipy.sparse.csr_matrix.toarray

We can also index and slice into the sparse matrix.

In [ ]:
print('A dense representation like we saw in the slides.')
print(bow.toarray())
print()

In [ ]:
print('Indexing')
print(bow[0])
print()

In [ ]:
print('Indexing')
print(bow[1])

In [ ]:
print('Slicing')
print(bow[0:2])

## 3) Cosine Similarity

Cosine similarity quantifies the similarity between two vectors by measuring the cosine of the angle between them. This is particularly useful in text analysis, where texts are converted into vectors. Each dimension of the vector represents a word from the document, with its value indicating the frequency or importance of that word.

When calculating cosine similarity, first, the dot product of the two vectors is found. This product gives a measure of how vectors in the same direction are aligned. Then, the magnitudes (or lengths) of each vector are calculated. The cosine similarity is the dot product divided by the product of the two vectors' magnitudes.

This method effectively captures the orientation (or direction) of the vectors and not their magnitude, making it a reliable measure of similarity in texts of varying lengths. It's widely used in applications like recommendation systems, document clustering, and information retrieval, where understanding the similarity or dissimilarity between texts is crucial.

There are multiple ways to calculate Cosine similarity using scipy:
<br><br>
One way is using the **spatial** package, which is a collection of spatial algorithms and data structures. It has a method to calculate cosine *distance*. To get the cosine *similarity*, we have to substract the distance from 1.<br>
https://docs.scipy.org/doc/scipy/reference/spatial.html<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html#scipy.spatial.distance.cosine

In [ ]:
corpus

In [ ]:
# The cosine method expects array_like inputs, so we need to generate arrays from our sparse matrix.
doc1_vs_doc2 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[1].toarray()[0])
doc1_vs_doc3 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[2].toarray()[0])
doc1_vs_doc4 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[3].toarray()[0])

print(f"Doc 1 vs Doc 2: {doc1_vs_doc2}")
print(f"Doc 1 vs Doc 3: {doc1_vs_doc3}")
print(f"Doc 1 vs Doc 4: {doc1_vs_doc4}")

Another approach is using scikit-learn's *cosine_similarity* which computes the metric between multiple vectors. Here, we pass it our BOW and get a matrix of cosine similarities between each document.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
# cosine_similarity can take either array-likes or sparse matrices.
print(cosine_similarity(bow))

In [ ]:
print(cosine_similarity(bow[0:2]))

## 4) N-grams

**CountVectorizer** includes an *ngram_range* parameter to generate different n-grams. n_gram range is specified using a minimum and maximum range. By default, n_gram range is set to (1, 1) which generates unigrams. Setting it to (1, 2) generates both unigrams and bigrams.

In [ ]:
corpus

In [ ]:
# Both unigrams and bigrams
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, lowercase=False, binary=True, ngram_range=(1,2))
bigrams = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names_out(),"\n")

print('Number of features: {}'.format(len(vectorizer.get_feature_names_out())),"\n")

print(vectorizer.vocabulary_)

In [ ]:
#print(bigrams)

In [ ]:
# Setting n_gram range to (2, 2) generates only bigrams.
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, lowercase=False, binary=True, ngram_range=(2,2))
bigrams = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out(),"\n")
print(vectorizer.vocabulary_)

## Basic Bag-of-Words Exercises

In [ ]:
# EXERCISE: Create a spacy_tokenizer callback which takes a string and returns
# a list of tokens (each token's text) with punctuation filtered out.

corpus = [
  "Students use their GPS-enabled cellphones to take birdview photographs of a land in order to find specific danger points such as rubbish heaps.",
  "Teenagers are enthusiastic about taking aerial photograph in order to study their neighbourhood.",
  "Aerial photography is a great way to identify terrestrial features that aren’t visible from the ground level, such as lake contours or river paths.",
  "During the early days of digital SLRs, Canon was pretty much the undisputed leader in CMOS image sensor technology.",
  "Syrian President Bashar al-Assad tells the US it will 'pay the price' if it strikes against Syria."
]

nlp = spacy.load('en_core_web_sm')

def spacy_tokenizer(doc):
    return [t.text for t in nlp(doc) if not t.is_punct]

vectorizer = CountVectorizer(tokenizer=spacy_tokenizer)

bow = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names_out())

In [ ]:
# The string below is a whole paragraph. We want to create another # binary BOW but using the vocabulary
# of our *current* CountVectorizer. This means that words in this paragraph which AREN'T already in the
# vocabulary won't be represented. This is to illustrate how BOW can't handle out-of-vocabulary words
# unless you rebuild your whole vocabulary. Still, we'll see that if there's enough overlapping vocabulary,
# some similarity can still be picked up.


In [ ]:
# Note that we call 'transform' only instead of 'fit_transform' because the
# fit step (i.e. vocabulary build) is already done and we don't want to re-fit here.

s = ["Teenagers take aerial shots of their neighbourhood using digital cameras sitting in old bottles which are launched via kites - a common toy for children living in the favelas. They then use GPS-enabled smartphones to take pictures of specific danger points - such as rubbish heaps, which can become a breeding ground for mosquitoes carrying dengue fever."]
new_bow = vectorizer.transform(s)

# EXERCISE: using the pairwise cosine_similarity method from sklearn, calculate the similarities between
# each document from the corpus against this new document (new_bow).
# HINT: You can pass two parameters to cosine_similarity in this case. See the docs:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html#scipy.spatial.distance.cosine

# Which document is the most similar? Which is the least similar? Do the results make sense based on what you see?
print(corpus)
print()
print(s)

In [ ]:
cosine_similarity(new_bow, bow[0])

In [ ]:
cosine_similarity(new_bow, bow[1])

In [ ]:
cosine_similarity(new_bow, bow[2])

In [ ]:
cosine_similarity(new_bow,bow[3])

In [ ]:
# All in one statement
cosine_similarity(new_bow, bow[0:4])

In [ ]:
# EXERCISE: In spacy_tokenizer, instead of returning the plain text, return the lemma_ attribute instead.
# How do the cosine similarity results differ? What if you filter out stop words as well?

# B) TF-IDF



`TF-IDF` stands for Term Frequency Inverse Document Frequency of records. It can be defined as the calculation of how relevant a word in a series or corpus is to a text. The meaning increases proportionally to the number of times in the text a word appears but is compensated by the word frequency in the corpus (data-set).

**Term frequency (TF)**: number of times a term has appeared in a document. The term frequency is a measure of how frequently or how common a word is for a given sentence.

**Inverse document frequency (IDF)**: is a measure of how rare a word is in a document. Words like “the”,” a” show up in all the documents but rare words will not occur in all the documents of the corpus.

If a word appears in almost every document means it is not significant for the classification.






In [ ]:
#!pip install -U spacy==3.*
#!python -m spacy download en_core_web_sm
#!python -m spacy info

In [ ]:
import spacy

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Fetching datasets

This time around, rather than using a short toy corpus, let's use a larger dataset. scikit-learn has a **datasets** module with utilties to load datasets of our own as well as fetch popular reference datasets online.<br>
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets
<br><br>
We'll use the **20 newsgroups** dataset, which is a collection of 18,000 newsgroup posts across 20 topics.<br>
https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset
<br><br>
List of datasets available:<br>
https://scikit-learn.org/stable/datasets.html#datasets

The **datasets** module includes fetchers for each dataset in scikit-learn. For our purposes, we'll fetch only the posts from the *sci.space* topic, and skip on headers, footers, and quoting of other posts.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups
<br><br>
By default, the fetcher retrieves the *training* subset of the data only. If you don't know what that means, it'll become clear later in the course when we discuss modelling. For now, it doesn't matter for our purposes.

In [ ]:
corpus = fetch_20newsgroups(categories=['sci.space'],
                            remove=('headers', 'footers', 'quotes'))

In [ ]:
#corpus

We get back a **Bunch** container object containing the data as well as other information.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html
<br><br>
The actual posts are accessed through the *data* attribute and is a list of strings, each one representing a post.

In [ ]:
print(type(corpus))

In [ ]:
# Number of posts in our dataset.
len(corpus.data)

In [ ]:
# View first post.
corpus.data[0]

In [ ]:
# View 2nd and 3rd post.
corpus.data[1:3]

## Creating TF-IDF features

In [ ]:
# Like before, if we want to use spaCy's tokenizer, we need
# to create a callback. Remember to upgrade spaCy if you need
# to (refer to beginnning of file for commentary and instructions).
nlp = spacy.load('en_core_web_sm')
print(nlp,"\n")

# We don't need named-entity recognition nor dependency parsing for
# this so these components are disabled. This will speed up the
# pipeline. We do need part-of-speech tagging however.
unwanted_pipes = ["ner", "parser"]

# For this exercise, we'll remove punctuation and spaces (which
# includes newlines), filter for tokens consisting of alphabetic
# characters, and return the lemma (which require POS tagging).
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            not t.is_punct and \
            not t.is_space and \
            t.is_alpha]

spacy_tokenizer

Like the classes to create raw frequency and binary bag-of-words vectors, scikit-learn includes a similar class called **TfidfVectorizer** to create TF-IDF vectors from a corpus.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
<br><br>
The usage pattern is similar in that we call *fit_transform* on the corpus which generates the vocabulary dictionary (fit step), and generates the TF-IDF vectors (transform step).

In [ ]:
# Use the default settings of TfidfVectorizer.
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
vectorizer

In [ ]:
features = vectorizer.fit_transform(corpus.data)
features

In [ ]:
# The number of unique tokens.
print(len(vectorizer.get_feature_names_out()))

In [ ]:
# The dimensions of our feature matrix. X rows (documents) by Y columns (tokens).
print(features.shape)

In [ ]:
# What the encoding of the first document looks like in sparse format.
print(features[0])

As we mentioned in the slides, there are TF-IDF variations out there and scikit-learn, among other things, adds **smoothing** (adds a one to the numerator and denominator in the IDF component), and normalizes by default. These can be disabled if desired using the *smooth_idf* and *norm* parameters respectively. See here for more information:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


## Querying the data

The similarity measuring techniques we learned previously can be used here in the same way. In effect, we can query our data using this sequence:
1. *Transform* our query using the same vocabulary from our *fit* step on our corpus.
2. Calculate the pairwise cosine similarities between each document in our corpus and our query.
3. Sort them in descending order by score.

In [ ]:
# Transform the query into a TF-IDF vector.
query = ["lunar orbit"]
query_tfidf = vectorizer.transform(query)
print(query_tfidf)

In [ ]:
# Calculate the cosine similarities between the query and each document.
# We're calling flatten() here becaue cosine_similarity returns a list of lists and we just want a single list.
cosine_similarities = cosine_similarity(features, query_tfidf).flatten()
#cosine_similarities

Now that we have our list of cosine similarities, we can use this utility function to return the indices of the top k documents with the highest cosine similarities.

In [ ]:
import numpy as np

# numpy's argsort() method returns a list of *indices* that would sort an array:
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
# The sort is ascending, but we want the largest k cosine_similarites at the bottom of the sort.
# So we negate k, and get the last k entries of the indices list in reverse order.
# There are faster ways to do this using things like argpartition but this is more succinct.

def top_k(arr, k):
  kth_largest = (k + 1) * -1
  return np.argsort(arr)[:kth_largest:-1]

In [ ]:
# The top five documents.
top_related_indices = top_k(cosine_similarities, 5)
print(top_related_indices)

In [ ]:
# Let's take a look at their respective cosine similarities.
print(cosine_similarities[top_related_indices])

In [ ]:
# Top match.
print(corpus.data[top_related_indices[0]])

In [ ]:
# Second-best match.
print(corpus.data[top_related_indices[1]])

In [ ]:
# Try a different query
query = ["satellite"]
query_tfidf = vectorizer.transform(query)
print(query_tfidf)

cosine_similarities = cosine_similarity(features, query_tfidf).flatten()
top_related_indices = top_k(cosine_similarities, 5)

print(top_related_indices)
print(cosine_similarities[top_related_indices])

In [ ]:
print(corpus.data[top_related_indices[0]])

So here we have the beginnings of a simple search engine but we're a far cry from competing with commercial off-the-shelf search engines, let alone Google.
<br>
- For each query, we're scanning through our entire corpus, but in practice, you'll want to create an **inverted index**. Search applications such as Elasticsearch do that under the hood.
- You'd also want to evaluate the efficacy of your search using metrics like **precision** and **recall**.
- Document ranking also tends to be more sophisticated, using different ranking functions like Okapi BM25. With major search engines, ranking also involves hundreds of variables such as what the user searched for previously, what do they tend to click on, where are they physically, and on and on. These variables are part of the "secret sauce" and are closely guarded by companies.
- Beyond word presence, intent and meaning are playing a larger role.
<br>

Information Retrieval is a huge, rich topic and beyond search, it's also key in tasks such as question-answering.

`Note:` sklearn provides 2 classes for implementing TF-IDF:
1. Tfidftransformer where we need to compute word counts then compute IDF values and then compute the TF-IDF scores.
2. Tfidfvectorizer here all the steps are done in a single step.

## TF-IDF Exercises

**EXERCISE**<br>
Read up on these concepts we just mentioned if you're curious.<br>

https://en.wikipedia.org/wiki/Inverted_index<br>
https://en.wikipedia.org/wiki/Precision_and_recall<br>
https://en.wikipedia.org/wiki/Okapi_BM25<br>

In [ ]:
# EXERCISE: fetch multiple topics from the 20 newsgroups dataset and query them using the approach we followed.
# A list of topics can be found here:
# https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset
#
# If you're feeling ambitious, incorporate n-grams or look at how you can measure precision and recall.